In [1]:
import casadi as ca

# Create an Opti instance
opti = ca.Opti()

#=========== Parameters of the ARX model 
a = 0.8
b  = 0.5
c = 0.1

#=========== Desired value of y 
yd = opti.parameter()
measured_outflow = opti.parameter()
measured_speed =  opti.parameter()

#=========== Define the Horizon
N = 10  

#=========== Declare Symbolic Variables (SISO but can be extended to MIMO or MISO)
outflow = opti.variable(N)
u = opti.variable(N)

#=========== Objective function
# Minimize the deviation of y from yd and control effort
objective = 0
for t in range(1, N):
    objective += 5*(outflow[t] - yd)**2 + 0.1*u[t]**2  

opti.minimize(objective)

# ARX model constraints
for t in range(1, N):
    opti.subject_to(outflow[t] == a * outflow[t-1] + b * u[t] + c)

# Additional constraints (e.g., on control input)
for t in range(N):
    opti.subject_to(u[t] >= 0)  # Lower bound on control input
    opti.subject_to(u[t] <= 5)   # Upper bound on control input

# Initial conditions
opti.subject_to(outflow[0] == measured_outflow)  # Initial value of y, read from sensor
opti.subject_to(u[0] == measured_speed)  # Initial value of u, read from sensor

# Solver options
opts = {
    "qpsol": "qrqp",
    #"convexify_strategy": "regularize",
    "print_time": 0,
    "print_iteration": 0,
    "print_header": 0,
    "verbose": False,
    "error_on_fail": False
}

opti.solver("sqpmethod", opts)


In [2]:
name = 'func'
func = opti.to_function(name, [yd, measured_outflow, measured_speed], [u, outflow, yd])

-------------------------------------------
This is casadi::QRQP
Number of variables:                              20
Number of constraints:                            31
Number of nonzeros in H:                          18
Number of nonzeros in A:                          49
Number of nonzeros in KKT:                       149
Number of nonzeros in QR(V):                     103
Number of nonzeros in QR(R):                     166


In [3]:
import os 
import os

os.chdir('c_code_generated')
opts = dict(main=True, \
            mex=False)
            # Move into the generated code directory
func.generate('edge_mpc.c',opts)
return_value = os.system('gcc -fPIC -shared edge_mpc.c -o edge_mpc.so -lm')
if return_value == 0:
    print("Code generation successful")
else:
    print("Code generation failed")


Code generation successful


In [4]:
f = ca.external('func', 'edge_mpc.so')

In [5]:
u_opt, y_opt, yref = f(10, 2, 1)